# Grundlagen

In diesem Abschnitt widmen wir uns einer praktischen SPARQL-Übung, indem wir uns mit der folgenden Forschungsfrage auseinandersetzen:

**Wie viele Datensätze bietet jedes deutsche Bundesland als offene Daten an?**

Wir arbeiten mit Jupyter Notebook - einem Dateiformat, welches ermöglicht, Erklärtext und Code in verschiedenen Programmiersprachen darzustellen. Dadurch wird die visuelle Darstellung von echtem Code für Abfragen und deren Ergebnisse ermöglicht. Zusätzlich können Erläuterungen zu den aufgerufenen Befehlen sowie weitere Kommentare dargestellt werden. 

Zuerst widmen wir uns der Analyse der SPARQL-Syntax, indem wir uns die Abfragen der Forschungsfragen ansehen. Als allererstes muss ein sogenannter "endpoint" definiert werden. Der endpoint ist die maschinelesbare Schnittstelle zum Repositorium, in dem jene Metadaten gespeichert sind, die wir abfragen wollen. Bei der Arbeit mit einem Online-SPARQL-Werkzeug ist die Definition eines endpoints häufig nicht nötig, da dieser schon automatisch definiert ist. Im Portal GOVDATA ist der endpoint die Schnittstelle zum deutschen Open Data Portal <a href="https://www.govdata.de/" class="external-link" target="_blank">govdata.de</a>

Beispielhaft wollen wir nun das europäische Datenportal durchsuchen, welches wir auf der Webseite <a href="https://data.europa.eu/de" class="external-link" target="_blank">https://data.europa.eu/de</a> finden. Anschließend verwenden wir folgende "Magic", d. h. einen Befehl, der Teil vom SPARQL-Python-Paket ist, der es uns erlaubt, alle künftigen Abfragen mit unserem *endpoint* zu verknüpfen.

**Code**

In [ ]:
%endpoint https://data.europa.eu/sparql

Endpoint set to: https://data.europa.eu/sparql

**Erklärung**<br>Mit dem festgelegten *endpoint* können wir SPARQL-Abfragen erstellen. Allerdings fragen wir nur jene Metadaten ab, welche zu den Datensätzen im Portal gespeichert sind. Siehe hierzu Kapitel ... . 

**Ergebnis**

Bevor wir mit der Abfrage von Metadaten beginnen, werden wir uns mit der Onthologie der gespeicherten Daten vertraut machen. Der im [Kapitel DCAT-AP](dcat-1) erklärte Metatenstandard <a href="https://www.dcat-ap.de/def/" class="external-link" target="_blank">DCAT-AP</a> ist zentral für die Untersuchung der Metadateneigentschaften und definiert die Struktur und den Inhalt der Metadatenfelder (<a href="https://www.dcat-ap.de/def/dcatde/2.0/implRules/" class="external-link" target="_blank">DCAT-AP Handbuch</a>).
Schauen wir uns nun die Struktur unserer ersten Abfrage an.

**Code**

```SPARQL
PREFIX dct: <http://purl.org/dc/terms/>                                    # (1)
PREFIX dcatde: <http://dcat-ap.de/def/dcatde/>                             # (1)
PREFIX pg: <https://www.dcat-ap.de/def/politicalGeocoding/>                # (1)

SELECT ?uri ?title ?contributorid ?stateKey                                # (2)
WHERE {                                                                    # (3)
    ?uri dct:title ?title .                                                # (3)
    ?uri dcatde:contributorID ?contributorid .                             # (3)
  OPTIONAL {?uri pg:stateKey ?stateKey} .                                  # (3)
}                                                                          # (3)
```

:::{admonition} Erklärung des Codes
:class: hinweis, dropdown

Als erstes sehen wir die sogenannten PREFIXes (siehe **(1)**). Die Prefixes sind ein nützliches Tool, das dabei hilft, auf diverse Eigenschaften zu verweisen und verkürzt Bezüge zwischen diesen Eigenschaften herzustellen. Sie sind zwar wichtig für eine einfachere Gestaltung der Abfragen, aber nicht essenziell. Sie helfen lediglich dabei, das Links nicht immer wieder ausgeschrieben werden müssen, und ermöglichen es nur die Endungen nach den Prefixes angeben zu müssen. Dies wird deutlicher, wenn wir uns den WHERE-Abschnitt ansehen.

Vorerst betrachten wir aber den SELECT-Befehl (siehe **(2)**). Mit SELECT wählen wir die Properties bzw. die Eigenschaften, die aufgelistet werden sollen. Jede Eigenschaft entspricht einer Spalte, die in der Tabelle mit Ergebnissen zu sehen ist. Da im SELECT-Befehl die folgenden 3 Properties ausgeschrieben werden - ?uri ?title ?contributorid ?stateKey - erhalten wir die URIs, die Titel, die Namen der Datenbereitsteller und das Kürzel des jeweiligen Landes, aus dem der Datensatz stammt.

Die genauen Benennungen der Properties (Labels) werden im DCAT-AP-Handbuch definiert, auf das wir zurückgreifen müssen, um die genauen Labels für jede Property zu finden.

Was Ihnen noch auffallen könnte ist, dass die Spalte für das Bundesland (stateKey) leer ist. Leider liegt das daran, dass das Land nicht codiert worden ist. Somit bleiben diese Felder leer. Dies ist ein klares Beispiel für lückenhaftes Metadatenmanagement, das die Beantwortung unserer Forschungsfrage erschwert. In der Praxis kommt es oft zu Fällen, in denen Abfragen nicht sehr erfolgreich sind, wegen unvollständigen Metadatenbeschreibungen. 

Als nächstes haben wir den Kern jeder SPARQL-Abfrage - den WHERE-Befehl (siehe **(3)**). Der WHERE-Befehl definiert die Beobachtungen, die aufgelistet werden sollen, indem die Bedingungen definiert werden. Somit werden nur die Beobachtungen aufgelistet, die alle Bedungungen erfüllen. In der Abfrage ist auch OPTIONAL zu sehen - dies besagt, dass die folgende Bedingung nicht zwingend zu erfüllen ist. Das bedeutet, dass selbst die Beobachtungen, in unserem Fall die Datensätze, in der Liste stehen, die keine Ausprägung für die Eigentschaft stateKey (Verweis auf Land) haben. Da leider stateKey nicht mit codiert ist, können wir uns alle Datensätze ansehen, die auf die anderen Bedingungen treffen (URI, Titel und ID der Bereitsteller), ohne dass wir eine leere Liste bekommen. OPTIONAL ist ein gutes Werkzeug, das benutzt werden kann, wenn man sich nicht sicher ist, ob jeweiligen Properties ordentlich codiert sind.

Was wahrscheinlich noch auffällt ist, dass in jeder Zeile in der WHERE-Funktion 3 Elemente zu sehen sind. Diese Struktur ist essentiell für die SPARQL-Sprache - durch die sogenannten "triplets" werden Bezüge zwischen den Eigenschaften erstellt. Jede Zeile bestimmt einen Bezug zwischen 2 Eigenschaften. Die erste Eigenschaft ist somit das Subjekt (S), das zweite Element - der Bezug, der aus einem Prefix und einer zusätzlichen Spezifizierung besteht, heißt das Prädikat (P), und das dritte - die zweite Eigenschaft, ist das Objekt (O). P entspricht einem Link, der darauf verweist, wo die zweite Eigentschaft zu finden ist. Die Einordnung der Eigenschaften ist nach dem W3C-Standard, der schon *in einem früheren Kapitel erklärt wurde*, definiert. In dem DCAT-AP-Handbuch ist dann die genaue Verortung von jeder Eingenschaft zu finden. Durch die Triplets fragen wir genau ab, welche Datensätze wir erfragen wollen, je nach den Bedingungen, die solche Datensätze erfüllen sollen. Mit unserer Abfrage suchen wir die Datensätze ab, die einen Titel, ein URI, einen mitcodierten Datenbereitsteller, und wenn vorhanden, einen Schlüssel für das Bundesland, haben, was leider bei keinem der Datensätze der Fall ist. Es lässt sich darauf schließen, dass diese Felder nicht verpflichtend ausgefüllt werden müssen. 

Wichtig zu bedenken ist, dass SPARQL leider keine Paginierungsfunktion unterstützt. Man muss es in der Regel auf Anwendungsebene handhaben, da SPARQL von sich aus nicht das Durchblättern von Ergebnissen wie eine Weboberfläche unterstützt. Stattdessen muss die Paginierung manuell durch die Verwendung von LIMIT und OFFSET in den Abfragen implementiert werden. Dies erfordert eine zusätzliche Logik in der Anwendung, um die aktuelle Seite Ausgabe vollständig zu sehen. 

Leider konnten wir unsere Fragestellung wegen mangelhafter Daten nicht ganz beantworten. Deshalb versuchen wir, unsere Fragestellung zu ändern und zusätzliche Beispiele von SPARQL-Abfragen damit aufzuzeigen.
:::

In [ ]:
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dcatde: <http://dcat-ap.de/def/dcatde/>
PREFIX pg: <https://www.dcat-ap.de/def/politicalGeocoding/>

SELECT ?uri ?title ?contributorid ?stateKey
WHERE {
    ?uri dct:title ?title .
    ?uri dcatde:contributorID ?contributorid .
  OPTIONAL {?uri pg:stateKey ?stateKey} .
}

uri,title,contributorid,stateKey
http://data.europa.eu/88u/dataset/b7cad163-8d99-4d97-b781-e4f5a11078d3,Digitale Topographische Karte 1 : 10 000 - 3952-NO Friedland - Groß Muckrow,http://dcat-ap.de/def/contributors/openDataBrandenburg,
http://data.europa.eu/88u/dataset/b7cad163-8d99-4d97-b781-e4f5a11078d3,Léarscáil thopagrafach dhigiteach 1 : 10 000 - 3952-NO Friedland - Groß Muckrow,http://dcat-ap.de/def/contributors/openDataBrandenburg,
http://data.europa.eu/88u/dataset/b7cad163-8d99-4d97-b781-e4f5a11078d3,Ψηφιακός τοπογραφικός χάρτης 1: 10000 — 3952-NO Friedland — Groß Muckrow,http://dcat-ap.de/def/contributors/openDataBrandenburg,
http://data.europa.eu/88u/dataset/b7cad163-8d99-4d97-b781-e4f5a11078d3,Digitalna topografska karta 1.: 10 000 - 3952-NO Friedland - Groß Muckrow,http://dcat-ap.de/def/contributors/openDataBrandenburg,
http://data.europa.eu/88u/dataset/b7cad163-8d99-4d97-b781-e4f5a11078d3,Digitālā topogrāfiskā karte Nr. 1 : 10 000 - 3952-NO Friedland - Groß Muckrow,http://dcat-ap.de/def/contributors/openDataBrandenburg,
http://data.europa.eu/88u/dataset/b7cad163-8d99-4d97-b781-e4f5a11078d3,Digitaalinen topografinen kartta 1 : 10 000 - 3952-NO Friedland - Groß Muckrow,http://dcat-ap.de/def/contributors/openDataBrandenburg,
http://data.europa.eu/88u/dataset/b7cad163-8d99-4d97-b781-e4f5a11078d3,Mapa topográfico digital 1 : 10 000 - 3952-NO Frísia - Groß Muckrow,http://dcat-ap.de/def/contributors/openDataBrandenburg,
http://data.europa.eu/88u/dataset/b7cad163-8d99-4d97-b781-e4f5a11078d3,Carta topografica digitale 1 : 10 000 - 3952-NO Friedland - Groß Muckrow,http://dcat-ap.de/def/contributors/openDataBrandenburg,
http://data.europa.eu/88u/dataset/b7cad163-8d99-4d97-b781-e4f5a11078d3,Digitálna topografická mapa 1 : 10 000 - 3952-NO Friedland - Groß Muckrow,http://dcat-ap.de/def/contributors/openDataBrandenburg,
http://data.europa.eu/88u/dataset/b7cad163-8d99-4d97-b781-e4f5a11078d3,Digitale topografische kaart 1 : 10 000 - 3952-NO Friedland - Groß Muckrow,http://dcat-ap.de/def/contributors/openDataBrandenburg,
